In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

- The folder for the data sets

In [2]:
DATA_PATH = Path("./datasets/")

- Importation of the data

In [3]:
df = pd.read_csv(DATA_PATH/"academy_awards.csv",
                 encoding="ISO-8859-1")

In [4]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['Year'] = df['Year'].str[:4].astype(int)

- We will select film later than 2000

In [6]:
later_than_2000 = df[df['Year'] > 2000].copy()

- Actor or Actress : leading  role or supporting role

In [7]:
award_categories = ["Actor -- Leading Role", "Actor -- Supporting Role",
                    "Actress -- Leading Role", "Actress -- Supporting Role"]

In [8]:
nominees = later_than_2000[later_than_2000['Category'].isin(award_categories)]

- Rename Won col

In [9]:
renaming_dict = {"YES": 1, "NO": 0}

In [10]:
new_name = nominees.loc[:, 'Won?'].copy().map(renaming_dict)

In [11]:
nominees['Won'] = new_name

c:\Users\maham\anaconda3\envs\package-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
col_to_drop = [col for col in nominees.columns if col.startswith("Unnamed")] + ["Won?"]

In [13]:
final_nominations = nominees.drop(col_to_drop, axis=1)

Additional info has it organization
- The name of the movie and the name of the actor in the film in curly bracket
- We will seperate the film and the name

In [14]:
additional_info = final_nominations['Additional Info'].str.rstrip("'}")

In [15]:
additional_info_splitted = additional_info.str.split("{'", expand=True)

In [16]:
movies_name = additional_info_splitted[0]
characters = additional_info_splitted[1]

In [17]:
final_nominations['Movies'] = movies_name
final_nominations['Characters'] = characters

In [18]:
final_nominations.drop('Additional Info', axis=1, inplace=True)

In [19]:
connection = sqlite3.connect(DATA_PATH/"nominations.db")

- We can play with if_exist params

In [ ]:
final_nominations.to_sql("nominations",connection, index=False)

In [23]:
for row in connection.execute("select * FROM nominations limit 10;").fetchall():
    print(row)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful ', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit ', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network ', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech ", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours ', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter ', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone ", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town ', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right ', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech ", 'Lionel Logue')


- Create ceremonies table

In [24]:
(2010, "Steve Martin"),
(2009, "Hugh Jackman"),
(2008, "John Stewart"),
(2007, "Ellen DeGeneres"),
(2006, "John Stewart"),
(2005, "Chris Rock"),
(2004, "Billy Crystal"),
(2003, "Steve Martin"),
(2002, "Whoopi Goldberg"),
(2001, "Steve Martin"),
(2000, "Billy Crystal")

(2000, 'Billy Crystal')

In [25]:
connection.execute("create table ceremonies(id integer primary key, Year integer, Host text);").fetchall()

[]

- Check all tables in our db

In [27]:
connection.execute("select name from sqlite_master where type=='table'").fetchall()

[('nominations',), ('ceremonies',)]

- Check the schema of the table

In [28]:
connection.execute("pragma table_info(ceremonies);").fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'Year', 'INTEGER', 0, None, 0),
 (2, 'Host', 'TEXT', 0, None, 0)]

In [29]:
year_hosts = [(2010, "Steve Martin"),
(2009, "Hugh Jackman"),
(2008, "John Stewart"),
(2007, "Ellen DeGeneres"),
(2006, "John Stewart"),
(2005, "Chris Rock"),
(2004, "Billy Crystal"),
(2003, "Steve Martin"),
(2002, "Whoopi Goldberg"),
(2001, "Steve Martin"),
(2000, "Billy Crystal")]

In [32]:
insert_query = "insert into ceremonies (Year, Host) values (?, ?);"

In [33]:
connection.executemany(insert_query, year_hosts).fetchall()

[]

In [34]:
connection.commit()

In [36]:
connection.execute("select * from ceremonies limit 10;").fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'John Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'John Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

- Link Foreign key with the db

In [37]:
connection.execute("pragma foreign_key = on;").fetchall()

[]

In [38]:
# One to many relation with nominations tables
connection.execute(f"create table nominations_two(id integer primary key, category text,"
                    f"nominee text, movie text, character text"
                   f", won int, ceremony_id int references ceremonies(id));").fetchall()

[]

In [43]:
connection.execute("pragma table_info(nominations);").fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movies', 'TEXT', 0, None, 0),
 (5, 'Characters', 'TEXT', 0, None, 0)]

In [46]:
join_nomination_query = f"""
select nominations.category, nominations.nominee,
       nominations.movies, nominations.characters, nominations.won, ceremonies.id
from nominations
inner join ceremonies
on nominations.year = ceremonies.year;
"""


In [47]:
joined_nomination = connection.execute(join_nomination_query).fetchall()

In [52]:
nomination_two_insertion_query = "insert into nominations_two values (null, ?, ?, ?, ?, ?, ?);"

In [53]:
connection.executemany(nomination_two_insertion_query, joined_nomination).fetchall()

[]

In [56]:
connection.commit()

In [ ]:
# delete the table nominations:drop table name_of_table
# rename: alter table old_name rename to new_name
connection.execute('drop table nominations;').fetchall()


In [ ]:
connection.execute('later table nominations_two rename into nominations;').fetchall()

In [ ]:
# Create movie_actors table
create_movie_query = "create table movies (id integer primary key, movie text)"
create_actor_query = "create table actors (id integer primary key, actor text)"

create_movie_actor_query = f"""CREATE TABLE movies_actors (id integer primary key,
        movie_id integer references movies(id), actor_id integer references actors(id));"""

In [58]:
connection.close()